<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/check/R_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [2]:
import pandas as pd

In [3]:
!gdown --id 1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S #df_all_raw
df_all = pd.read_csv('df_all_raw.csv')

Downloading...
From: https://drive.google.com/uc?id=1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S
To: /content/df_all_raw.csv
244MB [00:01, 138MB/s]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### NB the df_all to start should be the cleaned one. We use this to check if R code works

In [12]:
%%R # to run R on colab
#This script creates a data frame of colexifications based on the raw CLICS data.
#It then joins this data with d_cosines, a dataframe containing two columns of words/glosses and another one with their cosine_similarity
#Rows for which there is no cosine_sim value are dropped  (e.g., multi-word expressions or those that are OOV)
library(dplyr)
library(readr)
library(tidyr)


#if (!file.exists("df_colexifications_new.csv")){ #if the file doesnt exist: compute it
  df <- read_csv('df_all_raw.csv') #original raw CLICS file
  d_colex <- df %>%
    select(clics_form,Concepticon_ID,Glottocode,Concepticon_Gloss,Family,variety) %>% #--> added fam and var
    drop_na() %>%
    unique()

  dd <- inner_join(d_colex,d_colex,by=c("clics_form","Glottocode","Family","variety")) %>% #--> added fam and var
    filter(Concepticon_ID.x != Concepticon_ID.y) %>% # find rows/cases where conditions are true
    unique() # remove duplicates
  d_colex <- dd[!duplicated(lapply(as.data.frame(t(dd), stringsAsFactors=FALSE), sort)),]  
  #remove rows where Concepticon_ID1 and Concepticon_ID2 are permutations of each other 

  #Code colefixications with 1
  #d_colex$colex <- rep(1,nrow((d_colex)))

  write_csv(d_colex,'df_colexifications_new.csv')
#} else { #if it exists: load it
# print('Loading pre-processed colexification df')
  #d_colex <- read_csv('df_colexifications.csv')
#}

R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  Concepticon_ID = col_double(),
  Latitude = col_double(),
  Longitude = col_double(),
  AGE_OF_ACQUISITION = col_double(),
  CONCRETENESS = col_double(),
  FAMILIARITY = col_double(),
  IMAGABILITY = col_double(),
  KUCERA_FRANCIS_FREQUENCY = col_double()
)
ℹ Use `spec()` for the full column specifications.




In [9]:
%%R
d_colex

# A tibble: 224,576 x 10
   clics_form Concepticon_ID.x Glottocode Concepticon_Glo… Family.x variety.x
   <chr>                 <dbl> <chr>      <chr>            <chr>    <chr>    
 1 s@                     1369 hrus1242   GOLD             Hruso    Hruso Ak…
 2 s@                     1369 hrus1242   GOLD             Hruso    Hruso Ak…
 3 avir@                  1035 miji1239   GOOD             Sino-Ti… Dammai D…
 4 jang                   1425 chug1252   GREEN            Sino-Ti… Chug Par…
 5 dzju                   1425 hrus1242   GREEN            Hruso    Hruso Ak…
 6 tsam                   2648 kala1376   HAIR (HEAD)      Sino-Ti… Monpa Ba…
 7 tsam                   2648 kala1376   HAIR (HEAD)      Sino-Ti… Monpa Ba…
 8 khra                   2648 dakp1242   HAIR (HEAD)      Sino-Ti… Monpa Ch…
 9 khra                   2648 dakp1242   HAIR (HEAD)      Sino-Ti… Monpa Ch…
10 khra                   2648 dakp1242   HAIR (HEAD)      Sino-Ti… Monpa Ch…
# … with 224,566 more rows, and 4 more 

In [ ]:
%%R
dd

# A tibble: 351,398 x 8
   clics_form Concepticon_ID.x Glottocode Concepticon_Glo… Family variety
   <chr>                 <dbl> <chr>      <chr>            <chr>  <chr>  
 1 s@                     1369 hrus1242   GOLD             Hruso  Hruso …
 2 s@                     1369 hrus1242   GOLD             Hruso  Hruso …
 3 avir@                  1035 miji1239   GOOD             Sino-… Dammai…
 4 dzju                   1425 hrus1242   GREEN            Hruso  Hruso …
 5 tsam                   2648 kala1376   HAIR (HEAD)      Sino-… Monpa …
 6 khra                   2648 dakp1242   HAIR (HEAD)      Sino-… Monpa …
 7 khra                   2648 dakp1242   HAIR (HEAD)      Sino-… Monpa …
 8 tsam                   2648 kala1376   HAIR (HEAD)      Sino-… Monpa …
 9 khra                   2648 dakp1242   HAIR (HEAD)      Sino-… Monpa …
10 lah                    1277 dakp1242   HAND             Sino-… Monpa …
# … with 351,388 more rows, and 2 more variables: Concepticon_ID.y <dbl>,
#   Conceptico

following code is not useful now

In [ ]:
d_cosines <- read_csv('cosines.csv')

d_colex <- d_colex %>%
      mutate(Concepticon_Gloss.x = tolower(Concepticon_Gloss.x), #lowering glosses to match to pre-processing done for fasttext
      	     Concepticon_Gloss.y = tolower(Concepticon_Gloss.y)) %>%
      left_join(d_cosines, by=c('Concepticon_Gloss.x' = 'Concepticon_Gloss', #matching to fasttext
      	                       'Concepticon_Gloss.y' = 'Concepticon_Gloss_2')) %>%
      drop_na() #drop rows for which there is no cosine similarity (e.g., multi-word expressions or those that are OOV)

write_csv(d_colex,'df_colex_and_cosine.csv')

In [10]:
df_colex = pd.read_csv('df_colexifications_new.csv')


In [11]:
df_colex.head()

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family.x,variety.x,Concepticon_ID.y,Concepticon_Gloss.y,Family.y,variety.y
0,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,1927,BAMBOO,Hruso,Hruso Aka Jamiri
1,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,946,BLOOD,Hruso,Hruso Aka Jamiri
2,avir@,1035,miji1239,GOOD,Sino-Tibetan,Dammai Dibin,923,LOVE,Sino-Tibetan,Dammai Dibin
3,jang,1425,chug1252,GREEN,Sino-Tibetan,Chug Parchu,847,THIRSTY,Sino-Tibetan,Duhumbi
4,dzju,1425,hrus1242,GREEN,Hruso,Hruso Aka Jamiri,1424,YELLOW,Hruso,Hruso Aka Jamiri


In [ ]:
df_colex.info

<bound method DataFrame.info of        clics_form  Concepticon_ID.x  ... Concepticon_ID.y Concepticon_Gloss.y
0              s@              1369  ...             1927              BAMBOO
1              s@              1369  ...              946               BLOOD
2           avir@              1035  ...              923                LOVE
3            dzju              1425  ...             1424              YELLOW
4            tsam              2648  ...              492               THREE
...           ...               ...  ...              ...                 ...
175694        wus               763  ...               51                SORE
175695      mping              2458  ...             1257                WING
175696         su                51  ...             1257                WING
175697         ib              1040  ...              675  FAECES (EXCREMENT)
175698       kuji              1222  ...             1837               VULVA

[175699 rows x 8 columns]>

In [5]:
!gdown --id 1a-Bnhiwo3TArMAKKABETdie4flLTF5C1
df_colex_original = pd.read_csv('df_colexifications.csv')

Downloading...
From: https://drive.google.com/uc?id=1a-Bnhiwo3TArMAKKABETdie4flLTF5C1
To: /content/df_colexifications.csv
7.64MB [00:00, 46.5MB/s]


In [ ]:
df_colex_original.head()

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Concepticon_ID.y,Concepticon_Gloss.y,colex
0,s@,1369,hrus1242,GOLD,1927,BAMBOO,1
1,s@,1369,hrus1242,GOLD,946,BLOOD,1
2,avir@,1035,miji1239,GOOD,923,LOVE,1
3,jang,1425,chug1252,GREEN,847,THIRSTY,1
4,dzju,1425,hrus1242,GREEN,1424,YELLOW,1


In [7]:
df_colex_original.info

<bound method DataFrame.info of        clics_form  Concepticon_ID.x  ... Concepticon_Gloss.y colex
0              s@              1369  ...              BAMBOO     1
1              s@              1369  ...               BLOOD     1
2           avir@              1035  ...                LOVE     1
3            jang              1425  ...             THIRSTY     1
4            dzju              1425  ...              YELLOW     1
...           ...               ...  ...                 ...   ...
182063          n              2473  ...               CHILD     1
182064         kv              2400  ...                 OUR     1
182065       kura              2112  ...               BELLY     1
182066          i               780  ...                  ME     1
182067       nono              2099  ...         CALF OF LEG     1

[182068 rows x 7 columns]>

PROBLEM: the number of rows from the original df_colexification and the modified one is different, why??
even if we run the code without modifications, the number changes

we found that the number of varieties is different with the number of glottocode: problem with inner_join?